<a href="https://colab.research.google.com/github/VrayZ2384/Music-Mood-Visualizer/blob/main/Music_MultiModal_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/openai/CLIP.git
#https://github.com/openai/CLIP
#CLIP (Contrastive Language-Image Pre-Training)
#Learning Transferable Visual Models From Natural Language Supervision
#Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh,
#Sandhini Agarwal, Girish Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever

!git clone https://github.com/CompVis/taming-transformers
#https://github.com/CompVis/taming-transformers
#Taming Transformers for High-Resolution Image Synthesis
#Patrick Esser, Robin Rombach, Björn Ommer


In [ ]:
## install some extra libraries
!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
!pip uninstall torchtext --yes
!pip install einops
!pip install librosa
!pip install pytube
!pip install requests
!pip install flask
!pip install flask-cors
!pip install tqdm
# Install ngrok
!pip install pyngrok


In [3]:
# import libraries
import numpy as np
import torch, os, imageio, pdb, math
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import librosa
import pytube
from pyngrok import ngrok
import shutil
from http.server import SimpleHTTPRequestHandler
import http.server
import socketserver
import json
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Image, display
from PIL import Image as PILImage
import requests
from io import BytesIO
import PIL
import matplotlib.pyplot as plt
from tqdm import tqdm

import yaml
from omegaconf import OmegaConf

from CLIP import clip

import warnings
warnings.filterwarnings('ignore')

In [4]:
## helper functions

def show_from_tensor(tensor):
  img = tensor.clone()
  img = img.mul(255).byte()
  img = img.cpu().numpy().transpose((1,2,0))

  plt.figure(figsize=(10,7))
  plt.axis('off')
  plt.imshow(img)
  plt.show()

def norm_data(data):
  return (data.clip(-1,1)+1)/2 ### range between 0 and 1 in the result

### Parameters
learning_rate = .5
batch_size = 1
wd = .1
noise_factor = .22

total_iter=400
im_shape = [450, 450, 3] # height, width, channel
size1, size2, channels = im_shape

In [ ]:
### CLIP MODEL ###
clipmodel, _ = clip.load('ViT-B/32', jit=False)
clipmodel.eval()
print(clip.available_models())

print("Clip model visual input resolution: ", clipmodel.visual.input_resolution)

device=torch.device("cuda:0")
torch.cuda.empty_cache()

In [ ]:
## Taming transformer instantiation

%cd taming-transformers/

!mkdir -p models/vqgan_imagenet_f16_16384/checkpoints
!mkdir -p models/vqgan_imagenet_f16_16384/configs

if len(os.listdir('models/vqgan_imagenet_f16_16384/checkpoints/')) == 0:
   !wget 'https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1' -O 'models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt'
   !wget 'https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1' -O 'models/vqgan_imagenet_f16_16384/configs/model.yaml'

In [ ]:
from taming.models.vqgan import VQModel

def load_config(config_path, display=False):
   config_data = OmegaConf.load(config_path)
   if display:
     print(yaml.dump(OmegaConf.to_container(config_data)))
   return config_data

def load_vqgan(config, chk_path=None):
  model = VQModel(**config.model.params)
  if chk_path is not None:
    state_dict = torch.load(chk_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
  return model.eval()

def generator(x):
  x = taming_model.post_quant_conv(x)
  x = taming_model.decoder(x)
  return x

taming_config = load_config("./models/vqgan_imagenet_f16_16384/configs/model.yaml", display=True)
taming_model = load_vqgan(taming_config, chk_path="./models/vqgan_imagenet_f16_16384/checkpoints/last.ckpt").to(device)

In [8]:
### Declare the values that we are going to optimize

class Parameters(torch.nn.Module):
  def __init__(self):
    super(Parameters, self).__init__()
    self.data = .5*torch.randn(batch_size, 256, size1//16, size2//16).cuda() # 1x256x14x15 (225/16, 400/16)
    self.data = torch.nn.Parameter(torch.sin(self.data))

  def forward(self):
    return self.data

def init_params():
  params=Parameters().cuda()
  optimizer = torch.optim.AdamW([{'params':[params.data], 'lr': learning_rate}], weight_decay=wd)
  return params, optimizer


In [ ]:
def MusicMood(song_path):
  # Load the audio file
  audio, sr = librosa.load(song_path)

  # Extract audio features
  tempo, beats = librosa.beat.beat_track(y=audio, sr=sr)
  chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
  mfcc = librosa.feature.mfcc(y=audio, sr=sr)

  # Analyze the features and extract the music mood
  mood = analyzeMusicMood(tempo, beats, chroma, mfcc)

  return mood

def analyzeMusicMood(tempo, beats, chroma, mfcc):
    # Ensure that tempo is a NumPy array
    if not isinstance(tempo, np.ndarray):
        tempo = np.array([tempo])

    # Calculate the average tempo and beat variance
    avg_tempo = tempo.mean()
    beat_var = beats.var()

    # Calculate the chroma and MFCC mean values
    chroma_mean = chroma.mean(axis=1)
    mfcc_mean = mfcc.mean(axis=1)

    # Based on the calculated features, determine the music mood
    if avg_tempo > 120 and beat_var < 100 and chroma_mean[3] > 0.5:
        mood = 'Happy'  # Happy
    elif avg_tempo < 100 and chroma_mean[1] > 0.3 and mfcc_mean[5] > -5:
        mood = 'Sad'  # Sad
    else:
        mood = 'Neutral'  # Neutral

    return mood


### Encoding prompts and a few more things
normalize = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

def encodeText(text):
  t=clip.tokenize(text).cuda()
  t=clipmodel.encode_text(t).detach().clone()
  return t

def createEncodings(include, exclude, extras):
  include_enc=[]
  for text in include:
    include_enc.append(encodeText(text))
  exclude_enc=encodeText(exclude) if exclude != '' else 0
  extras_enc=encodeText(extras) if extras !='' else 0

  return include_enc, exclude_enc, extras_enc

augTransform = torch.nn.Sequential(
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomAffine(30, (.2, .2), fill=0)
).cuda()

Params, optimizer = init_params()

with torch.no_grad():
  print(Params().shape)
  img= norm_data(generator(Params()).cpu()) # 1 x 3 x 224 x 400 [225 x 400]
  print("img dimensions: ",img.shape)
  show_from_tensor(img[0])


In [10]:
### create crops

def create_crops(img, num_crops=32):
  p=size1//2
  img = torch.nn.functional.pad(img, (p,p,p,p), mode='constant', value=0) # 1 x 3 x 448 x 624 (adding 112*2 on all sides to 224x400)

  img = augTransform(img) #RandomHorizontalFlip and RandomAffine

  crop_set = []
  for ch in range(num_crops):
    gap1= int(torch.normal(1.2, .3, ()).clip(.43, 1.9) * size1)
    offsetx = torch.randint(0, int(size1*2-gap1),())
    offsety = torch.randint(0, int(size1*2-gap1),())

    crop=img[:,:,offsetx:offsetx+gap1, offsety:offsety+gap1]

    crop = torch.nn.functional.interpolate(crop,(224,224), mode='bilinear', align_corners=True)
    crop_set.append(crop)

  img_crops=torch.cat(crop_set,0) ## 30 x 3 x 224 x 224

  randnormal = torch.randn_like(img_crops, requires_grad=False)
  num_rands=0
  randstotal=torch.rand((img_crops.shape[0],1,1,1)).cuda() #32

  for ns in range(num_rands):
    randstotal*=torch.rand((img_crops.shape[0],1,1,1)).cuda()

  img_crops = img_crops + noise_factor*randstotal*randnormal

  return img_crops


In [11]:
### Show current state of generation

def showme(Params, show_crop):
  with torch.no_grad():
    generated = generator(Params())

    if (show_crop):
      aug_gen = generated.float() # 1 x 3 x 224 x 400
      aug_gen = create_crops(aug_gen, num_crops=1)
      aug_gen_norm = norm_data(aug_gen[0])
      #show_from_tensor(aug_gen_norm)

    latest_gen=norm_data(generated.cpu()) # 1 x 3 x 224 x 400
    show_from_tensor(latest_gen[0])

  return (latest_gen[0])


In [12]:
# Optimization process

def optimize_result(Params, prompt):
  alpha=1 ## the importance of the include encodings
  beta=.5 ## the importance of the exclude encodings

  ## image encoding
  out = generator(Params())
  out = norm_data(out)
  out = create_crops(out)
  out = normalize(out) # 30 x 3 x 224 x 224
  image_enc=clipmodel.encode_image(out) ## 30 x 512

  ## text encoding  w1 and w2
  final_enc = w1*prompt + w1*extras_enc # prompt and extras_enc : 1 x 512
  final_text_include_enc = final_enc / final_enc.norm(dim=-1, keepdim=True) # 1 x 512
  final_text_exclude_enc = exclude_enc

  ## calculate the loss
  main_loss = torch.cosine_similarity(final_text_include_enc, image_enc, -1) # 30
  penalize_loss = torch.cosine_similarity(final_text_exclude_enc, image_enc, -1) # 30

  final_loss = -alpha*main_loss + beta*penalize_loss

  return final_loss

def optimize(Params, optimizer, prompt):
  loss = optimize_result(Params, prompt).mean()
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  return loss



In [23]:
# Create a tqdm progress bar for the entire training process
def training_loop(Params, optimizer, show_crop=False):
    res_img = []
    res_z = []

    total_iterations = len(include_enc) * total_iter
    current_iteration = 0

    # Create a tqdm progress bar for the entire training process
    total_progress_bar = tqdm(total=total_iterations, desc="Generating")

    for prompt in include_enc:
        iteration = 0
        Params, optimizer = init_params()  # Initialize Params and optimizer for each prompt

        for it in range(total_iter):
            loss = optimize(Params, optimizer, prompt)

            if iteration >= 80 and iteration % show_step == 0:
                new_img = showme(Params, show_crop)
                res_img.append(new_img)
                res_z.append(Params())  # 1 x 256 x 14 x 25
                print("loss:", loss.item(), "\niteration:", iteration)

            current_iteration += 1
            total_progress_bar.update(1)  # Update the total progress bar

            iteration += 1

    # Close the total progress bar
    total_progress_bar.close()

    torch.cuda.empty_cache()
    return res_img, res_z


In [ ]:
print("=======================================")
print("|  Welcome to the Song Mood Visualizer |")
print("| This program generates visuals based |")
print("|   on the mood of a song.             |")
print("=======================================")

# Function to generate images
def generate_images(event):
    clear_output()

    # Get the YouTube link from the text input widget
    youtube_link = song_link_input.value

    # Create a YouTube object and get the best audio stream
    yt = pytube.YouTube(youtube_link)
    audio_stream = yt.streams.filter(only_audio=True).first()

    # Get the song's title and artist name from YouTube
    song_title = yt.title
    artist_name = yt.author

    # Download the audio stream to a file
    audio_stream.download(output_path="/content", filename=f"{song_title}.mp3")

    # Set the path to the downloaded MP3 file
    song_path = f"/content/{song_title}.mp3"

    # Display song information
    print("=======================================")
    print(f"  Song Information                   ")
    print(f"  Title: {song_title}                ")
    print(f"  Artist: {artist_name}              ")
    print("=======================================")

    # Get the cover image of the song from YouTube and display it
    response = requests.get(yt.thumbnail_url)
    img = PILImage.open(BytesIO(response.content))
    display(img)

    include = []
    # Determine the mood of the song
    song_mood = MusicMood(song_path)
    print("\n\n=======================================")
    print(f"  Mood: {song_mood}                      ")
    print("=======================================")

    if song_mood == 'Happy':  # Happy mood
            include.extend([
                f'Create a joyful and lively scene that matches the happy mood of the song "{song_title}"',
                f'song "{song_title}"',
                f'Generate a vibrant and colorful representation of happiness in the song "{song_title}".'
            ])
    elif song_mood == 'Sad':  # Sad mood
        include.extend([
            f'Capture the melancholic and somber atmosphere that resonates with the sad mood of the song "{song_title}" ',
            f'song "{song_title}',
            f'Create a quiet and reflective scene that captures the sadness in the song "{song_title}".'
        ])
    elif song_mood == 'Neutral':  # Neutral mood
        include.extend([
            f'Craft a calm and peaceful landscape that reflects the neutral mood of the song "{song_title}"',
            f'song "{song_title}"',
            f'Generate a serene and tranquil environment that mirrors the song\'s neutral emotions in "{song_title}".'
    ])

    global include_enc, exclude_enc, extras_enc, w1, w2, noise_factor, total_iter, show_step, res_z, res_img

    # Define include_enc, exclude_enc, and extras_enc based on prompts
    exclude = 'watermark, cropped, confusing, incoherent, cut, blurry'
    extras = 'concise, detailed, high-quality, creative, realistic, surreal, historical, futuristic, mystical, educational, humorous'
    w1 = 1
    w2 = 1
    noise_factor = 0.22
    total_iter = 110
    show_step = 20  # set this to see the result every 20 iterations beyond iteration 80

    include_enc, exclude_enc, extras_enc = createEncodings(include, exclude, extras)

    # Now you can call training_loop
    res_img, res_z = training_loop(Params, optimizer, show_crop=True)

    # Display images or results here
    display("Images generated successfully!")  # You can replace this with displaying images

# Create a text input widget for the YouTube link
song_link_input = widgets.Text(
    value='',
    placeholder='Enter YouTube link',
    description='YouTube Link:',
    disabled=False
)

# Create a button widget for generating images
generate_button = widgets.Button(
    description='Generate Images',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate images',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Attach the click handler to the button
generate_button.on_click(generate_images)

# Display widgets
display(song_link_input, generate_button)

In [38]:
def interpolate(res_z_list, duration_list):
  gen_img_list=[]
  fps = 25

  for idx, (z, duration) in enumerate(zip(res_z_list, duration_list)):
    num_steps = int(duration*fps)
    z1=z
    z2=res_z_list[(idx+1)%len(res_z_list)] # 1 x 256 x 14 x 25 (225/16, 400/16)

    for step in range(num_steps):
      alpha = math.sin(1.5*step/num_steps)**6
      z_new = alpha * z2 + (1-alpha) * z1

      new_gen=norm_data(generator(z_new).cpu())[0] ## 3 x 224 x 400
      new_img=T.ToPILImage(mode='RGB')(new_gen)
      gen_img_list.append(new_img)

  return gen_img_list

durations=[5,5,5,5,5,5]
interp_result_img_list = interpolate(res_z, durations)

In [39]:
## create a video
out_video_path=f"../video.mp4"
writer = imageio.get_writer(out_video_path, fps=25)
for pil_img in interp_result_img_list:
  img = np.array(pil_img, dtype=np.uint8)
  writer.append_data(img)

writer.close()

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('../video.mp4','rb').read()
data="data:video/mp4;base64,"+b64encode(mp4).decode()
HTML("""<video width=800 controls><source src="%s" type="video/mp4"></video>""" % data)